In [1]:
# Import necessary libraries and frameworks

import numpy as np
import tensorflow as tf
import random
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from tensorflow import keras
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import time
from modules.model_architecture import overall

In [2]:
# Load the model architecture and set it up with specific parameters

model = overall(
    text_size = 35,
    audio_size = 10,
    vision_size = 5,
    game_size = 5,
    text_dim = 768,
    audio_dim = 65,
    vision_dim = 128,
    game_dim = 40,
    kernel_size = 1,
    num_layers = 3,
    dff = 128,
    d_model = 32,
    num_heads = 8,
    dropout = 0.1,
    output_bias = tf.keras.initializers.Constant(np.log([3.247]))
)

# NOTE: Ensure the model file is available in the repository or provide instructions on how to obtain it.
model_path = "path_to_your_model.h5"  # Replace with your model path or placeholder
model.load_weights(model_path)

# Compile the model with focal loss and metrics
METRICS = [
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.AUC(name='auc')
]
focal_loss = tf.keras.losses.BinaryFocalCrossentropy(apply_class_balancing=True, alpha=3, gamma=2)
model.compile(loss=focal_loss, optimizer=tf.keras.optimizers.Adam(0.00005), metrics=METRICS)
model.summary()

Model: "overall"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 audio_input (InputLayer)       [(None, 10, 65)]     0           []                               
                                                                                                  
 text_input (InputLayer)        [(None, 35, 768)]    0           []                               
                                                                                                  
 vision_input (InputLayer)      [(None, 5, 128)]     0           []                               
                                                                                                  
 game_input (InputLayer)        [(None, 5, 40)]      0           []                               
                                                                                            

 )                                                                                                
                                                                                                  
 bidirectional_5 (Bidirectional  (None, 64)          57600       ['audio_fusion[0][0]']           
 )                                                                                                
                                                                                                  
 bidirectional_6 (Bidirectional  (None, 64)          57600       ['vision_fusion[0][0]']          
 )                                                                                                
                                                                                                  
 bidirectional_7 (Bidirectional  (None, 64)          57600       ['game_fusion[0][0]']            
 )                                                                                                
          

## Method 1: Save the entire model + TFLite

In [3]:
model.save('my_model')

INFO:tensorflow:Assets written to: my_model\assets


INFO:tensorflow:Assets written to: my_model\assets


In [4]:
converter = tf.lite.TFLiteConverter.from_saved_model('my_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
converter._experimental_default_to_single_batch_in_tensor_list_ops = True
tflite_model = converter.convert()

In [5]:
# Save the TFLite model
# NOTE: Ensure the 'tflite/' directory is present in your repository or provide instructions to create it.

tflite_models_dir = "tflite/"
tflite_model_file = os.path.join(tflite_models_dir, "my_model.tflite")
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

## Method 2: Load model saved with weights + TFLite

In [ ]:
# NOTE: Ensure the model file is available in the repository or provide instructions on how to obtain it.

converter = tf.lite.TFLiteConverter.from_keras_model(model_path)  # Replace with your model path or placeholder
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter._experimental_default_to_single_batch_in_tensor_list_ops = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

In [ ]:
# Save the TFLite model
# NOTE: Ensure the 'tflite_model/' directory is present in your repository or provide instructions to create it.

tflite_models_dir = "tflite_model/"
tflite_model_file = os.path.join(tflite_models_dir, "my_model.tflite")
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

## Model Prediction

In [6]:
interpreter = tf.lite.Interpreter(model_path="tflite/my_model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [7]:
# Display input details

input_details

[{'name': 'serving_default_vision_input:0',
  'index': 0,
  'shape': array([  1,   5, 128]),
  'shape_signature': array([ -1,   5, 128]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'serving_default_audio_input:0',
  'index': 1,
  'shape': array([ 1, 10, 65]),
  'shape_signature': array([-1, 10, 65]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'serving_default_text_input:0',
  'index': 2,
  'shape': array([  1,  35, 768]),
  'shape_signature': array([ -1,  35, 768]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dty

In [8]:
# Display output details

output_details

[{'name': 'StatefulPartitionedCall:0',
  'index': 5180,
  'shape': array([1, 1]),
  'shape_signature': array([-1,  1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [9]:
# Set random input data for demonstration

input_shape_vision = input_details[0]['shape']
input_shape_audio = input_details[1]['shape']
input_shape_text = input_details[2]['shape']
input_shape_game = input_details[3]['shape']

input_vision = np.array(np.random.random_sample(input_shape_vision), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_vision)
input_audio = np.array(np.random.random_sample(input_shape_audio), dtype=np.float32)
interpreter.set_tensor(input_details[1]['index'], input_audio)
input_text = np.array(np.random.random_sample(input_shape_text), dtype=np.float32)
interpreter.set_tensor(input_details[2]['index'], input_text)
input_game = np.array(np.random.random_sample(input_shape_game), dtype=np.float32)
interpreter.set_tensor(input_details[3]['index'], input_game)

In [10]:
# Invoke the interpreter and get the output

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data[0][0])

0.43394184
